# Jupyter and IPython Configuration Tips

## Embedded Dependency Installation

### Overview
To install the requirements of a notebook from within a code cell, you can use the `%pip` magic available with *IPython* `0.7.3` and up (in earlier versions, use `!pip` instead). This also makes sure that installing uncommon packages used in your notebook is documented, improving reproducability.

Installed packages go into `~/.ipython`, but not directly into that directory, which would become a mess otherwise. Instead, we install into the usual `lib/pythonX.X/site-packages` hierarchy, and then make that visible to Python. An additional important advantage is that kernels using different Python versions co-exist peacefully.

`%env` magic can be used to point `pip` at a local repository server like *devpi* or *Artifactory*.

### Install & Import Mechanics
The following code is pure boilerplate to provide the `require` function. In production environments, you probably want to have this in your underlying configuration, or as a custom `%require` magic.

In [1]:
%env PIP_INDEX_URL=https://pypi.org/pypi
%env PIP_PREFIX=~/.ipython

def require(spec):
    """Helper for importing custom packages."""
    import importlib, os, re, sys

    site_packages = os.path.expanduser(
        '~/.ipython/lib/python{v.major}.{v.minor}/site-packages'
        .format(v=sys.version_info))
    if site_packages not in sys.path:
        sys.path.insert(0, site_packages)

    name = re.split('[ ;,<>=]', spec)[0].replace('-', '_')
    try:
        module = importlib.import_module(name)
        print("⚠ Using already installed '{}' package."
              .format(name))
    except ImportError:
        pip_opts = '-q --disable-pip-version-check --no-warn-script-location'
        %pip install {pip_opts} "{spec}"
        module = importlib.import_module(name)

    globals()[name] = module
    return module

env: PIP_INDEX_URL=https://pypi.org/pypi
env: PIP_PREFIX=~/.ipython


This code shows the extended Python search path with the user-specific `site-packages` directory upfront.

In [2]:
def prettify(path):
    import os
    return path.replace(os.path.expanduser('~/'), '~/')

require('sys')
[prettify(x) for x in sys.path]

⚠ Using already installed 'sys' package.


['~/.ipython/lib/python3.6/site-packages',
 '/opt/venvs/jupyterhub/lib/python36.zip',
 '/opt/venvs/jupyterhub/lib/python3.6',
 '/opt/venvs/jupyterhub/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '',
 '/opt/venvs/jupyterhub/lib/python3.6/site-packages',
 '/opt/venvs/jupyterhub/lib/python3.6/site-packages/IPython/extensions',
 '~/.ipython']

### Defining & Using requirements
Given the above code is executed once in a kernel, you can now replace a normal import with a `require` call, and the specified package is installed if not available yet.

In [3]:
require('distro>=1.4')
prettify(repr(distro))

Note: you may need to restart the kernel to use updated packages.


"<module 'distro' from '~/.ipython/lib/python3.6/site-packages/distro.py'>"

Finally, just use the package as usual.

In [4]:
distro.info()

{'id': 'ubuntu',
 'version': '18.04',
 'version_parts': {'major': '18', 'minor': '04', 'build_number': ''},
 'like': 'debian',
 'codename': 'bionic'}